# Finance-Python

> 原始项目地址：**Finance-Python**（https://github.com/wegamekinglc/Finance-Python）；

> ``python setup.py install`` 安装包；

> 相关依赖请见主目录下 ``requirements`` 文件夹。

## Operator in Declarative Style

### 声明式

计算表达式被抽象为一些**算子**，用户无需给出计算的流程，只需要使用这些**算子**表达自己的想法。

例如，我们需要计算向量 $\bar x$ 的均值，**命令式**的做法可能如下：

```python
sum_value = 0
for v in x:
   sum_value += v 
average_value = sum_value / len(x)
```

而**声明式**的做法：

```python
average_value = mean(x)
```

上面的 ``mean`` 就是我们所谓的算子概念。

### 延迟计算


表达式无需在定义的位置获得结果，只需在需要的时候进行计算即可：

```python

expression = operator(dependency)

```

中间会有一些其他计算的代码：

```
......
```

这里的 ``expression`` 是一个表达式对象，而不是计算的结果，在需要值的时候：

```python

expression_value = expression.value

```

### 支持算术运算

表达式对象支持基本的 ``+-*/`` 运算。

例如，计算收益率的sharp值，我们可以这样去定义表达式：

```python

sharp_expression = mean(x) / std(x)

```

这里的 ``sharp_expression`` 是一个新的表达式，由两个基础的表达式构造而成。

### 支持复合运算

运算可以复合，例如实现以下的逻辑，计算过去20日每日50日均线的标准差：

```python

compounded_expression = std(mean(x, 50), 20)

```

### 支持聚合以及投影

#### 聚合

可以多个表达式的结果，传入另一个依赖多个输入的表达式，例如：

$$
z = f(x, y) \\\\
x = g(a) \\\\
y = h(b)
$$

可以有组合表达式：

$$ z = f(g(a), h(b))$$

以伪代码的形式表达如下：

```python

exp1 = operator1(a)
exp2 = operator2(b)

exp3 = operator3(exp1, exp2)

```

#### 投影

有些时候我们需要把一个表达式一部分结果传入另一个表达式：

$$
    x, y = f(a) \\\\
    z = h(x)
$$

所以我们有下面的复合表达式

$$
    z = h([f(a)]_x)
$$

其中 $ [\cdot]_x$ 代表在 $x$ 方向上的投影。

如果以伪代码的形式：

```python

exp1 = operator1(a)

exp2 = operator2(exp1[0])

```

## Implementation in Finance-Python

在 **Finance-Python** 中，以 **accumulator** 的形式实现了上面的 **Declarative Style Operator**。**accumulator** 是具有自身状态的算符，

## Accumulator

### Hello World

下面的这个例子，使用 ``Latest`` 算符，保留输入值的最近状态。

In [ ]:
from PyFin.Math.Accumulators import Latest

exp1 = Latest(dependency='x')
exp1

上面可以看到 ```exp1``` 是一个 ``accumulator`` 的实例。

In [ ]:
# 1st round
exp1.push({'x': 1})
print("Value after 1st round: {0}".format(exp1.value))

# 2nd round
exp1.push({'x': 2})
print("Value after 2nd round: {0}".format(exp1.value))

# repeate
print("Do nothing: {0}".format(exp1.value))

# 3rd and 4th round
exp1.push({'x': 3})
exp1.push({'x': 4})
print("Value after 3rd/4th round: {0}".format(exp1.value))

### One Step Further

下面的例子，计算过去两个输入值的均值：

In [ ]:
from PyFin.Math.Accumulators import MovingAverage

ma = MovingAverage(dependency='x', window=2)

values = [1, 2, 3, 4, 5]

for i, x in enumerate(values):
    ma.push({'x': x})
    print("{0}: {1}".format(i, ma.value))

### More complicated examples

#### 算术运算/复合运算

计算一组收益率序列的滚动sharp，时间窗口为250日。

构造算子：

In [ ]:
import numpy as np
from PyFin.Math.Accumulators import MovingVariance
from PyFin.Math.Accumulators import Sqrt

np.random.seed(47)
ret_simulated = 0.0005 + np.random.randn(2000) / 100.

ret_mean = MovingAverage(dependency='x', window=250)
ret_std = Sqrt(MovingVariance(dependency='x', window=250)) # Compounded accumlator is used here

sharp = ret_mean / ret_std # dividing can be used for accumulators
sharp

输入数据：

In [ ]:
sharp_series = []
for ret in ret_simulated:
    sharp.push({'x': ret})
    sharp_series.append(sharp.value)

把数据画出来出来：

In [ ]:
%pylab inline
import seaborn as sns
import pandas as pd
sns.set_style('whitegrid')
df = pd.DataFrame({'returns': ret_simulated.cumsum(), 'sharp': sharp_series})
df[250:].plot(secondary_y='sharp', figsize=(12, 6))

#### 聚合运算

``Correlation`` 的计算需要涉及两个变量序列的计算，这里就有**聚合**。

下面的例子中，我们计算一组序列，20日均线和50日均线的250日相关系数：

In [ ]:
from PyFin.Math.Accumulators import MovingCorrelation

ma20 = MovingAverage(dependency='x', window=20)
ma50 = MovingAverage(dependency='x', window=50)

corr = MovingCorrelation(dependency=ma20^ma50, window=250)
corr

注意上面的代码中，我们使用 ``^`` 运算符表示了 **聚合** 运算。

In [ ]:
ma20_series = []
ma50_series = []
corr_series = []
for ret in ret_simulated:
    ma20.push({'x': ret})
    ma50.push({'x': ret})
    corr.push({'x': ret})
    
    ma20_series.append(ma20.value)
    ma50_series.append(ma50.value)
    corr_series.append(corr.value)

In [ ]:
df = pd.DataFrame({'ma20': ma20_series, 'ma50': ma50_series, 'corr': corr_series})
df[300:].plot(secondary_y='corr', figsize=(12, 6))

#### 投影运算

下面的例子，我们计算上面收益日序列，250日alpha值的20日均值。

在 ``accumulator`` 中，我们有 ``MovingAlphaBeta`` 计算过去一段时间的 $\alpha$ 以及 $\beta$ 值。因为这里我们只需要计算 $\alpha$ 的均值，这里我们就需要使用 **投影** 运算，只将部分数据输入 ``MovingAverage`` ：

In [ ]:
from PyFin.Math.Accumulators import MovingAlphaBeta

exp1 = MovingAlphaBeta(window=250)
alpha_mean = MovingAverage(dependency=exp1[0], window=20)

我们直接使用了 ``[]`` 做了投影的操作。

In [ ]:
alpha_series = []
alpha_mean_series = []
for ret in ret_simulated:
    exp1.push({'pRet': ret, 'mRet': 0., 'riskFree': 0.})
    alpha_mean.push({'pRet': ret, 'mRet': 0., 'riskFree': 0.})
    alpha_series.append(exp1.value[0])
    alpha_mean_series.append(alpha_mean.value)

In [ ]:
df = pd.DataFrame({'alpha': alpha_series, 'alpha20': alpha_mean_series})
df[250:].plot(secondary_y='alpha20', figsize=(12, 6))

### Working with Pandas

In [ ]:
from PyFin.api import MA
from PyFin.examples.datas import sample_data

In [ ]:
sample_data

In [ ]:
ma2 = MA(2, 'close')

#### 根据 ``category_field`` 计算

In [ ]:
ma2.transform(sample_data, category_field='code')

#### 直接计算

In [ ]:
ma2 = MA(2, 'close')

In [ ]:
ma2.transform(sample_data)